<a href="https://colab.research.google.com/github/jesse-code-sample/othello/blob/main/othello_code_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Game

In [1]:
import copy
from enum import IntEnum
from typing import List, Optional, Tuple


class Player(IntEnum):
    """Represents a player in Othello: BLACK (1) or WHITE (-1)."""
    BLACK = 1
    WHITE = -1

    def opponent(self) -> 'Player':
        """
        Returns the opposing player.

        Returns:
            Player: The opponent player.
        """
        return Player(-self.value)


class Piece(IntEnum):
    """
    Represents the content of a board cell:
    - BLACK (1): occupied by Black
    - WHITE (-1): occupied by White
    - NO_PIECE (0): empty
    """
    BLACK = 1
    NO_PIECE = 0
    WHITE = -1

    @staticmethod
    def from_player(player: Player) -> 'Piece':
        """
        Converts a Player into the corresponding Piece.

        Args:
            player (Player): The player.

        Returns:
            Piece: The matching piece.
        """
        return Piece(player.value)


class Game:
    """
    Class representing an Othello game state and rules.
    Handles piece placement, legal move checks, and board updates.
    """
    BOARD_SIZE = 8
    DIRS = [
        (-1, 0), (1, 0), (0, 1), (0, -1),
        (-1, 1), (1, 1), (-1, -1), (1, -1)
    ]

    def __init__(self, board: Optional[List[List[Piece]]] = None) -> None:
        """
        Initializes the Othello game board.

        Args:
            board (Optional[List[List[Piece]]]): An optional custom board state.
        """
        if board is not None:
            self.board: List[List[Piece]] = copy.deepcopy(board)
        else:
            self.board = [[Piece.NO_PIECE] * self.BOARD_SIZE for _ in
                          range(self.BOARD_SIZE)]
            self.board[3][3] = Piece.WHITE
            self.board[4][4] = Piece.WHITE
            self.board[3][4] = Piece.BLACK
            self.board[4][3] = Piece.BLACK

    def place_piece(self, player: Player, row: int, col: int) -> None:
        """
        Places a piece for the given player at the specified coordinates
        and flips opponent pieces.

        Args:
            player (Player): The current player.
            row (int): 0-based row index.
            col (int): 0-based column index.

        Raises:
            Exception: If the move is not legal.
        """
        if not self.is_legal_move(player, row, col):
            raise ValueError(f'Illegal move at row {row}, col {col}')
        piece = Piece.from_player(player)
        self.board[row][col] = piece
        for dx, dy, end_x, end_y in self._get_endpoints(player, row, col):
            x, y = row + dx, col + dy
            while (x, y) != (end_x, end_y):
                self.board[x][y] = piece
                x += dx
                y += dy

    def is_legal_move(self, player: Player, row: int, col: int) -> bool:
        """
        Determines if placing a piece at (row, col) is legal for the player.

        Args:
            player (Player): The current player.
            row (int): Row index.
            col (int): Column index.

        Returns:
            bool: True if the move is legal, False otherwise.
        """
        return (
                self.valid_coordinate(row, col)
                and self.board[row][col] == Piece.NO_PIECE
                and self._get_endpoints(player, row, col)
        )

    def valid_coordinate(self, row: int, col: int) -> bool:
        """
        Checks whether a board coordinate is within valid bounds.

        Args:
            row (int): Row index.
            col (int): Column index.

        Returns:
            bool: True if within bounds, False otherwise.
        """
        return 0 <= row < self.BOARD_SIZE and 0 <= col < self.BOARD_SIZE

    def _get_endpoints(self, player: Player, row: int, col: int) -> List[
        Tuple[int, int, int, int]]:
        """
        Finds all endpoints where placing a piece at the specified coordinates
        would flip opponent pieces between the coordinate and that endpoint.

        Args:
            player (Player): The current player.
            row (int): Row index.
            col (int): Column index.

        Returns:
            List[Tuple[int, int, int, int]]: A list of tuples containing the
            direction (dx, dy) and endpoint (end_x, end_y) for each valid
            flip.
        """
        endpoints = []
        for dx, dy in self.DIRS:
            x, y = row + dx, col + dy
            sandwiched = False
            while (self.valid_coordinate(x, y) and
                   self.board[x][y].value == player.opponent().value):
                sandwiched = True
                x += dx
                y += dy
            if (sandwiched and self.valid_coordinate(x, y) and
                    self.board[x][y].value == player.value):
                endpoints.append((dx, dy, x, y))
        return endpoints

    def get_legal_moves_for_player(self, player: Player) -> List[
        Tuple[int, int]]:
        """
        Returns all legal move positions for the given player.

        Args:
            player (Player): The player to check.

        Returns:
            List[Tuple[int, int]]: List of (row, col) tuples.
        """
        return [
            (row, col)
            for row in range(self.BOARD_SIZE)
            for col in range(self.BOARD_SIZE)
            if self.is_legal_move(player, row, col)
        ]

    def get_num_pieces(self) -> Tuple[int, int]:
        """
        Returns the number of placed black and white pieces.

        Returns:
            Tuple[int, int]: The number of placed (black_pieces, white_pieces)
        """
        black_pieces = white_pieces = 0
        for row in range(self.BOARD_SIZE):
            for col in range(self.BOARD_SIZE):
                if self.board[row][col] == Piece.BLACK:
                    black_pieces += 1
                elif self.board[row][col] == Piece.WHITE:
                    white_pieces += 1
        return black_pieces, white_pieces

    def clone(self) -> 'Game':
        """
        Returns a deep copy of the current game state.

        Returns:
            Game: A new Game instance with the same board.
        """
        return Game(self.board)

    def print_grid(self) -> None:
        """
        Prints the current board state to the console in a human-readable format.
        """
        print("\n    A   B   C   D   E   F   G   H\n")
        for i, row in enumerate(self.board):
            print(f"{i + 1}  ", end="")
            for cell in row:
                if cell == Piece.NO_PIECE:
                    print("--- ", end="")
                elif cell == Piece.BLACK:
                    print(" B  ", end="")
                else:
                    print(" W  ", end="")
            print("\n")

In [3]:
import unittest


class TestOthelloGame(unittest.TestCase):

    def setUp(self):
        self.game = Game()

    def test_initial_board(self):
        self.assertEqual(self.game.board[3][3], Piece.WHITE)
        self.assertEqual(self.game.board[4][4], Piece.WHITE)
        self.assertEqual(self.game.board[3][4], Piece.BLACK)
        self.assertEqual(self.game.board[4][3], Piece.BLACK)

    def test_valid_moves(self):
        self.assertTrue(self.game.is_legal_move(Player.BLACK, 2, 3))
        self.assertTrue(self.game.is_legal_move(Player.BLACK, 4, 5))
        self.assertTrue(self.game.is_legal_move(Player.WHITE, 2, 4))
        self.assertTrue(self.game.is_legal_move(Player.WHITE, 3, 5))

    def test_invalid_moves(self):
        self.assertFalse(self.game.is_legal_move(Player.WHITE, 0, 0))
        self.assertFalse(
            self.game.is_legal_move(Player.WHITE, 3, 3))  # Occupied

    def test_invalid_placement(self):
        with self.assertRaises(ValueError):
            self.game.place_piece(Player.WHITE, 3, 3)  # Already occupied
        with self.assertRaises(ValueError):
            self.game.place_piece(Player.WHITE, -1, 0)  # Out of bounds
        with self.assertRaises(ValueError):
            self.game.place_piece(Player.WHITE, 2, 3)  # No white piece to flip

    def test_flip_one_piece(self):
        """
        W B 'W'  ->  W W W
        B W          B W
        """
        self.game.place_piece(Player.WHITE, 3, 5)
        self.assertEqual(self.game.board[3][5], Piece.WHITE)
        self.assertEqual(self.game.board[3][4],
                         Piece.WHITE)  # Flipped from BLACK

    def test_flip_multiple_pieces(self):
        """
        W B B B 'W'  ->  W W W W W
        B W              B W
        """
        self.game.board[3][5] = self.game.board[3][6] = Piece.BLACK
        self.game.place_piece(Player.WHITE, 3, 7)
        for col in range(3, 8):
            self.assertEqual(self.game.board[3][col], Piece.WHITE)

    def test_flip_two_directions(self):
        """
        W B B 'W'  ->  W W W W
        B B B          B B W
          W              W
        """
        self.game.board[3][5] = Piece.BLACK
        self.game.board[4][4] = Piece.BLACK
        self.game.board[4][5] = Piece.BLACK
        self.game.board[5][4] = Piece.WHITE
        self.game.place_piece(Player.WHITE, 3, 6)
        for col in range(3, 7):
            self.assertEqual(self.game.board[3][col], Piece.WHITE)
        self.assertEqual(self.game.board[4][5], Piece.WHITE)

unittest.main(argv=['first-arg-is-ignored'], exit=False)

.......
----------------------------------------------------------------------
Ran 7 tests in 0.012s

OK


# Bot

In [4]:
import operator
from abc import abstractmethod, ABC
from typing import Optional, Tuple


class MinimaxBot(ABC):
    """
    A simple AI for Othello using minimax with alpha-beta pruning.
    """

    def __init__(self, depth: int) -> None:
        """
        Initializes the bot with a search depth.

        Args:
            depth (int): Depth of the minimax search tree.
        """
        self.depth = depth

    def get_best_move(self, game: Game, player: Player) -> Optional[
        Tuple[int, int]]:
        """
        Returns the best move for the given player.

        Args:
            game (Game): The current game state.
            player (Player): The player to make the move.

        Returns:
            Optional[Tuple[int, int]]: The (row, col) of the best move, or
            None if no legal moves.
        """
        _, move = self._minimax(game, player, self.depth, float('-inf'),
                                float('inf'))
        return move

    @abstractmethod
    def evaluate(self, game: Game) -> int:
        """
        Evaluates the game state. Positive values should indicate a more
        favorable evaluation for BLACK, while negative values should indicate a
        more favorable evaluation for WHITE.

        Args:
            game (Game): The current game state.

        Returns
            int: The evaluation of the game state.
        """
        pass

    def _minimax(
            self,
            game: Game,
            player: Player,
            depth: int,
            alpha: float,
            beta: float,
    ) -> Tuple[int, Optional[Tuple[int, int]]]:
        """
        Minimax algorithm with alpha-beta pruning. Maximizes the evaluation for
        the BLACK player, and minimizes the evaluation for the WHITE player.

        Args:
            game (Game): Current game state.
            player (Player): Player for whom to evaluate.
            depth (int): Remaining depth to search.
            alpha (float): Alpha pruning value.
            beta (float): Beta pruning value.

        Returns:
            Tuple[int, Optional[Tuple[int, int]]]: Evaluation score and move.
        """
        legal_moves = game.get_legal_moves_for_player(player)
        if depth == 0 or not legal_moves:
            return self.evaluate(game), None
        best_move = None
        best_eval = -float('inf') if player == Player.BLACK else float('inf')
        comparator = operator.gt if player == Player.BLACK else operator.lt
        for row, col in legal_moves:
            new_game = game.clone()
            new_game.place_piece(player, row, col)
            cur_eval, _ = self._minimax(new_game, player.opponent(), depth - 1,
                                        alpha, beta)
            if comparator(cur_eval, best_eval):
                best_eval = cur_eval
                best_move = (row, col)
            if player == Player.BLACK:
                alpha = max(alpha, cur_eval)
            else:
                beta = min(beta, cur_eval)
            if beta <= alpha:
                break
        return best_eval, best_move


class GreedyBot(MinimaxBot):
    """A simple bot that maximizes the difference in number of pieces."""

    @staticmethod
    def coin_score(game: Game) -> int:
        black_pieces, white_pieces = game.get_num_pieces()
        return black_pieces - white_pieces

    def evaluate(self, game: Game) -> int:
        return self.coin_score(game)


class CornersBot(GreedyBot):
    """A more advanced bot that prioritizes placing pieces on corners."""

    @staticmethod
    def corners_score(game: Game) -> int:
        corners_indices = [
            (0, 0),
            (0, game.BOARD_SIZE - 1),
            (game.BOARD_SIZE - 1, 0),
            (game.BOARD_SIZE - 1, game.BOARD_SIZE - 1),
        ]
        return sum(
            game.board[row][col] for row, col in corners_indices
        )

    def evaluate(self, game: Game) -> int:
        return self.coin_score(game) + 25 * self.corners_score(game)


def play_game(black_bot: MinimaxBot, white_bot: MinimaxBot,
              print_moves: bool = False) -> Tuple[int, int]:
    """
    Play an Othello game between the two provided bots.

    Args:
        black_bot (MinimaxBot): The bot playing black pieces.
        white_bot (MinimaxBot): The bot playing white pieces.
        print_moves (bool): Whether to print the game state and moves made
                            by the bots.

    Returns:
        Tuple[int, int]: The number of black pieces and white pieces at the end
                         of the game.
    """
    game = Game()
    if print_moves:
        game.print_grid()
    bots = [black_bot, white_bot]
    cur_bot, cur_player = 0, Player.BLACK
    while move := bots[cur_bot].get_best_move(game, cur_player):
        row, col = move
        game.place_piece(cur_player, row, col)
        if print_moves:
            print(
                f'{cur_player.name} moves at {row + 1}{chr(col + ord("A"))}'
            )
            game.print_grid()
        cur_bot = ~cur_bot
        cur_player = cur_player.opponent()
    return game.get_num_pieces()


In [7]:
import unittest


class TestMinimaxBot(unittest.TestCase):

    def test_depth(self):
        """
        Tests that a bot with greater search depth wins.
        Note that this may not be the case with GreedyBot, as it uses a poor
        heuristic.
        """
        shallow_bot = CornersBot(depth=1)
        deep_bot = CornersBot(depth=2)
        # Deep bot wins when going first
        black_pieces, white_pieces = play_game(black_bot=deep_bot,
                                               white_bot=shallow_bot)
        self.assertGreater(black_pieces, white_pieces)
        # Deep bot wins when going second
        black_pieces, white_pieces = play_game(black_bot=shallow_bot,
                                               white_bot=deep_bot)
        self.assertLess(black_pieces, white_pieces)

    def test_corners(self):
        """
        Tests that the CornersBot beats the GreedyBot.
        Note that this may not work at lower depths when GreedyBot goes first,
        as there is inherent first-player advantage.
        """
        greedy_bot = GreedyBot(depth=4)
        corners_bot = CornersBot(depth=4)
        # Corners bot wins when going first
        black_pieces, white_pieces = play_game(black_bot=corners_bot,
                                               white_bot=greedy_bot)
        self.assertGreater(black_pieces, white_pieces)
        # Corners bot wins when going second
        black_pieces, white_pieces = play_game(black_bot=greedy_bot,
                                               white_bot=corners_bot)
        self.assertLess(black_pieces, white_pieces)


unittest.main(argv=['first-arg-is-ignored'], exit=False)

.........
----------------------------------------------------------------------
Ran 9 tests in 26.993s

OK


# Bot vs Bot Demo

In [8]:
greedy_bot = GreedyBot(depth=4)
corners_bot = CornersBot(depth=4)
black_pieces, white_pieces = play_game(black_bot=greedy_bot,
                                        white_bot=corners_bot,
                                        print_moves=True)
print(
    f'Final piece count: {black_pieces} black pieces, '
    f'{white_pieces} white pieces'
)


    A   B   C   D   E   F   G   H

1  --- --- --- --- --- --- --- --- 

2  --- --- --- --- --- --- --- --- 

3  --- --- --- --- --- --- --- --- 

4  --- --- ---  W   B  --- --- --- 

5  --- --- ---  B   W  --- --- --- 

6  --- --- --- --- --- --- --- --- 

7  --- --- --- --- --- --- --- --- 

8  --- --- --- --- --- --- --- --- 

BLACK moves at 3D

    A   B   C   D   E   F   G   H

1  --- --- --- --- --- --- --- --- 

2  --- --- --- --- --- --- --- --- 

3  --- --- ---  B  --- --- --- --- 

4  --- --- ---  B   B  --- --- --- 

5  --- --- ---  B   W  --- --- --- 

6  --- --- --- --- --- --- --- --- 

7  --- --- --- --- --- --- --- --- 

8  --- --- --- --- --- --- --- --- 

WHITE moves at 3C

    A   B   C   D   E   F   G   H

1  --- --- --- --- --- --- --- --- 

2  --- --- --- --- --- --- --- --- 

3  --- ---  W   B  --- --- --- --- 

4  --- --- ---  W   B  --- --- --- 

5  --- --- ---  B   W  --- --- --- 

6  --- --- --- --- --- --- --- --- 

7  --- --- --- --- --- --- --- --- 

8  --

# Interactive Demo

In [11]:
def get_alpha_notation(row, col):
    return f'{row + 1}{chr(col + ord("A"))}'

In [13]:
game = Game()
game.print_grid()
bot = CornersBot(depth=4)
# Demo where you play the black pieces against a bot.
while True:
    # You have no available moves
    if bot.get_best_move(game, Player.BLACK) is None:
        break
    # Parse input into a move
    print('>', end=' ')
    input_str = input().strip()
    try:
        row = int(input_str[0]) - 1
        col = ord(input_str[1].upper()) - ord('A')
    except (IndexError, ValueError):
        print('Invalid input')
        continue
    if not game.is_legal_move(Player.BLACK, row, col):
        print(f'{input_str} is not a valid move')
        continue
    # Player's move
    print(f'You moved at: {get_alpha_notation(row, col)}')
    game.place_piece(Player.BLACK, row, col)
    game.print_grid()
    # Bot's move
    bot_move = bot.get_best_move(game, Player.WHITE)
    if bot_move is None:
        break
    print(f'AI moved at: {get_alpha_notation(*bot_move)}')
    game.place_piece(Player.WHITE, *bot_move)
    game.print_grid()

black_pieces, white_pieces = game.get_num_pieces()
print(
    f'Final piece count: {black_pieces} black pieces, '
    f'{white_pieces} white pieces'
)


    A   B   C   D   E   F   G   H

1  --- --- --- --- --- --- --- --- 

2  --- --- --- --- --- --- --- --- 

3  --- --- --- --- --- --- --- --- 

4  --- --- ---  W   B  --- --- --- 

5  --- --- ---  B   W  --- --- --- 

6  --- --- --- --- --- --- --- --- 

7  --- --- --- --- --- --- --- --- 

8  --- --- --- --- --- --- --- --- 

> 4C
You moved at: 4C

    A   B   C   D   E   F   G   H

1  --- --- --- --- --- --- --- --- 

2  --- --- --- --- --- --- --- --- 

3  --- --- --- --- --- --- --- --- 

4  --- ---  B   B   B  --- --- --- 

5  --- --- ---  B   W  --- --- --- 

6  --- --- --- --- --- --- --- --- 

7  --- --- --- --- --- --- --- --- 

8  --- --- --- --- --- --- --- --- 

AI moved at: 3C

    A   B   C   D   E   F   G   H

1  --- --- --- --- --- --- --- --- 

2  --- --- --- --- --- --- --- --- 

3  --- ---  W  --- --- --- --- --- 

4  --- ---  B   W   B  --- --- --- 

5  --- --- ---  B   W  --- --- --- 

6  --- --- --- --- --- --- --- --- 

7  --- --- --- --- --- --- --- --- 

8  